In [ ]:
from getData import *
get_data()

In [2]:
from data_preprocessing import *
from model import *
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

2024-05-11 00:53:35.935326: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 00:53:35.935363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 00:53:35.936025: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-11 00:53:35.940823: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = read_data_csv('data1.csv')
print(len(df.iloc[len(df)-24:]))

24


In [3]:
df.replace(to_replace=[-1.0], value=np.nan, inplace=True)
df

,CO2,PM10,PM25,NO,NO2,O3,SO2,AQI
0,495.0,-14.0,-123.0,2.00,2.00,3.00,2.00,2.0
1,400.0,15.0,12.7,0.94,42.16,8.58,33.38,1.0
2,400.0,15.0,12.7,0.94,42.16,8.58,33.38,1.0
3,400.0,15.0,12.6,0.94,42.16,8.58,33.38,1.0
4,413.0,15.1,12.7,0.94,42.16,8.58,33.38,1.0
...,...,...,...,...,...,...,...,...
1994,636.0,11.0,5.0,8.72,56.21,2.57,61.04,2.0
1995,635.0,8.7,4.7,8.72,56.21,2.57,61.04,2.0
1996,635.0,8.7,4.7,8.72,56.21,2.57,61.04,2.0
1997,606.0,9.1,4.8,8.72,56.21,2.57,61.04,2.0


In [ ]:
fill_missing_values(df)
df.drop(columns=['Unnamed: 0'], inplace=True)
df

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df)
print("X train: ", X_train.shape)
print("y train: ", y_train.shape)
print("X test: ", X_test.shape)
print("y test: ", y_test.shape)

X train:  (1599, 3)
y train:  (1599, 1)
X test:  (400, 3)
y test:  (400, 1)


In [6]:
# X_train_scaled, X_test_scaled = scale_data(X_train, X_test, ['PM25', 'PM10', 'CO2', 'NO', 'NO2', 'O3', 'SO2'])
X_train_scaled, X_test_scaled = scale_data(X_train, X_test, ['PM25', 'PM10', 'CO2'])

In [7]:
X_train_final, y_train_final = window_slide(X_train_scaled.values, y_train)
X_test_final, y_test_final = window_slide(X_test_scaled.values, y_test)
# print(y_train_final[:20])
onehot_labels_train = tf.keras.utils.to_categorical(y_train_final, num_classes=8)
onehot_labels_test = tf.keras.utils.to_categorical(y_test_final, num_classes=8)

print("X train: ", X_train_final.shape)
print("y train: ", onehot_labels_train.shape)
print("X test: ", X_test_final.shape)
print("y test: ", onehot_labels_test.shape)

X train:  (1589, 10, 3)
y train:  (1589, 8)
X test:  (390, 10, 3)
y test:  (390, 8)


In [8]:
model = build_lstm_autoencoder_model(X_train_final.shape[1:])
model.summary()

2024-05-10 14:51:08.893744: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 14:51:08.894093: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 14:51:08.960559: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 64)                9216      
 al)                                                             
                                                                 
 repeat_vector (RepeatVecto  (None, 3, 64)             0         
 r)                                                              
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                24832     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 8)                 520       
                                                                 
Total params: 34568 (135.03 KB)
Trainable params: 34568 (135.03 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [9]:
trained_model = train_model(model, X_train_final, onehot_labels_train, 0.0001, 50, 8)

Epoch 1/50


2024-05-10 14:51:17.427606: I external/local_xla/xla/service/service.cc:168] XLA service 0x55fecfb2e8d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-10 14:51:17.427639: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2024-05-10 14:51:17.427645: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-05-10 14:51:17.433283: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-10 14:51:17.448742: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1715352677.551808 3564484 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


199/199 [==============================] - 8s 27ms/step - loss: 2.0372 - accuracy: 0.4412
Epoch 2/50
199/199 [==============================] - 5s 27ms/step - loss: 1.7272 - accuracy: 0.4607
Epoch 3/50
199/199 [==============================] - 5s 27ms/step - loss: 1.0592 - accuracy: 0.6054
Epoch 4/50
199/199 [==============================] - 5s 26ms/step - loss: 0.8709 - accuracy: 0.6986
Epoch 5/50
199/199 [==============================] - 5s 26ms/step - loss: 0.7896 - accuracy: 0.7111
Epoch 6/50
199/199 [==============================] - 5s 27ms/step - loss: 0.7121 - accuracy: 0.7237
Epoch 7/50
199/199 [==============================] - 5s 27ms/step - loss: 0.6631 - accuracy: 0.7325
Epoch 8/50
199/199 [==============================] - 5s 26ms/step - loss: 0.6373 - accuracy: 0.7388
Epoch 9/50
199/199 [==============================] - 5s 26ms/step - loss: 0.6260 - accuracy: 0.7583
Epoch 10/50
199/199 [==============================] - 5s 26ms/step - loss: 0.6162 - accuracy: 0.7527


In [5]:
data = pd.read_csv("datatest2.csv")
X = df.iloc[:, :3]
    y = df['AQI']

    y = np.reshape(y, (y.shape[0], 1))

(476, 3)


ValueError: cannot reshape array of size 1428 into shape (476,10,3)

In [10]:
y_pred = trained_model.predict(X_test_final)
y_pred = np.argmax(y_pred, axis=1)

13/13 [==============================] - 0s 6ms/step


In [12]:
print(accuracy_score(y_test_final, y_pred))
print(classification_report(y_test_final, y_pred, labels=[1, 2, 3, 4, 5])) 

0.882051282051282
              precision    recall  f1-score   support

           1       0.92      0.80      0.85       125
           2       0.88      0.95      0.91       249
           3       0.67      0.50      0.57        16
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0

   micro avg       0.88      0.88      0.88       390
   macro avg       0.49      0.45      0.47       390
weighted avg       0.88      0.88      0.88       390



/home/jupyter-iec_haoquy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-iec_haoquy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-iec_haoquy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 

In [13]:
import pickle
# save the model to disk
filename = 'model1.h5'
pickle.dump(trained_model, open(filename, 'wb'))

In [14]:
trained_model.save('my_model.h5')

/opt/tljh/user/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
